In [3]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

reviews = getDF('/Users/taeyoungchoi/Documents/Fall 17/Personalization Theory/reviews_Video_Games.json.gz')
meta = getDF('/Users/taeyoungchoi/Documents/Fall 17/Personalization Theory/meta_Video_Games.json.gz')

In [4]:
print(reviews.columns.tolist())
print(meta.columns.tolist())
print(meta.shape)

['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime']
['asin', 'description', 'price', 'imUrl', 'related', 'salesRank', 'categories', 'title', 'brand']
(50953, 9)


In [5]:
print(meta.loc[313, ['asin', 'categories']])

asin               B00000J9J6
categories    [[Video Games]]
Name: 313, dtype: object


In [6]:
def assignment(item):
    wanted_categories = ['PC', 'Mac', 'More Systems', 'Nintendo', 'PlayStation', 'Sony', 'Video Games', 'Wii', 'Xbox']
    counter = 0
    for category in wanted_categories:
        if category in item:
            return wanted_categories[counter]
        else:
            counter += 1
    return ''


def assignment2(item):
    wanted_categories = ['PC', 'Mac', 'More Systems', 'Nintendo', 'PlayStation', 'Sony', 'Video Games', 'Wii', 'Xbox']
    counter = 0
    category = ''
    for category2 in wanted_categories:
        if category2 in item:
            category = category + " " + category2
            counter += 1
        else:
            counter += 1
    return category.strip()


def pick_categories(row):
    each_game_category = row['categories']
    category = ''
    if len(each_game_category) > 1:
        for i in range(len(each_game_category)):
            each_game = each_game_category[i]
            category = category + " " + each_game[1]
        return assignment2(category)
    else:
        if len(each_game_category[0]) == 1:
            return ''
        elif len(each_game_category[0]) > 3:
            return assignment(each_game_category[0][2])
        else:
            return assignment(each_game_category[0][1])
        
        
def mult_cat(row):
    each_game_category = row['cat']
    wanted_categories = ['PC', 'Mac', 'More Systems', 'Nintendo', 'PlayStation', 'Sony', 'Video Games', 'Wii', 'Xbox']
    counter = 0
    for category2 in wanted_categories:
        if category2 in each_game_category:
            counter += 1
    if counter >= 2:
        return "multi_support"
    else:
        return each_game_category

In [7]:
different_cat = meta.apply(lambda row: pick_categories(row), axis=1)
meta['cat'] = different_cat
meta2 = meta[meta['cat'] != '']
different_cat = meta2.apply(lambda row: mult_cat(row), axis=1)
meta['cat2'] = different_cat
cat_table = pd.crosstab(index = meta['cat2'], columns="count")
print(cat_table.sort_values(by='count', ascending=0))

In [10]:
meta_clean = meta[meta['cat2'].notnull()]
meta_clean = meta_clean[['asin', 'cat2']]
meta_clean.columns = ['asin', 'category']
merged_df = pd.merge(meta_clean, reviews, on = 'asin', how='inner')

In [34]:
import dateutil
merged_df['date'] = [dateutil.parser.parse(x) for x in merged_df['reviewTime']]
merged_df = merged_df.drop('reviewTime', 1)
print(merged_df.columns)
merged_df = merged_df.drop(merged_df.columns[[3, 4, 5, 7, 8]], 1)


In [52]:
relevant_data = merged_df[merged_df.date > '2011-01-01'] 
relevant_data.to_csv("clean_data.csv")

In [67]:
relevant_data.shape

(648665, 5)

In [68]:
print(len(set(relevant_data.asin)), len(set(relevant_data.reviewerID)))

31541 427533


In [81]:
relevant_data2 = relevant_data
relevant_data2 = relevant_data2.drop_duplicates(['asin'])
rows = relevant_data2.shape[0]
cat_prop = pd.crosstab(index = relevant_data2['category'], columns="count")
cat_prop['count'].div(rows).multiply(100)
## Mac 1 - Remove
## More Systems 12
## Nintendo 11
## PC 38
## PlayStation 13
## Sony 3
## Wii 7
## Xbox 10
## multi_support 6

category
Mac               1.309407
More Systems     11.692717
Nintendo         11.454932
PC               37.525760
PlayStation      12.710440
Sony              3.034146
Wii               6.597762
Xbox              9.815795
multi_support     5.859041
Name: count, dtype: float64

In [131]:
sample = {}
xbox = relevant_data[relevant_data['category'] == 'Xbox']
xbox_num_table = pd.crosstab(index = xbox['asin'], columns="count")
xbox_quant = list(xbox_num_table.quantile([.2,.4,.6,.8])['count'])

In [132]:
sample_id = xbox_num_table[xbox_num_table['count'] <= xbox_quant[0]].sample(n=2).index.tolist()
sample_id = sample_id + xbox_num_table[(xbox_num_table['count'] > xbox_quant[0]) & (xbox_num_table['count'] <= xbox_quant[1])].sample(n=2).index.tolist()
sample_id = sample_id + xbox_num_table[(xbox_num_table['count'] > xbox_quant[1]) & (xbox_num_table['count'] <= xbox_quant[2])].sample(n=2).index.tolist()
sample_id = sample_id + xbox_num_table[(xbox_num_table['count'] > xbox_quant[2]) & (xbox_num_table['count'] <= xbox_quant[3])].sample(n=2).index.tolist()
sample_id = sample_id + xbox_num_table[xbox_num_table['count'] > xbox_quant[3]].sample(n=2).index.tolist()
sample['xbox'] = sample_id

In [133]:
cat = relevant_data[relevant_data['category'] == 'More Systems']
cat_num_table = pd.crosstab(index = cat['asin'], columns="count")
quant = list(xbox_num_table.quantile([.25, .5, .75])['count'])
sample_id = cat_num_table[cat_num_table['count'] <= quant[0]].sample(n=3).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[0]) & (cat_num_table['count'] <= quant[1])].sample(n=3).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[1]) & (cat_num_table['count'] <= quant[2])].sample(n=3).index.tolist()
sample_id = sample_id + cat_num_table[cat_num_table['count'] > quant[2]].sample(n=3).index.tolist()
sample['More Systems'] = sample_id

In [134]:
cat = relevant_data[relevant_data['category'] == 'Nintendo']
cat_num_table = pd.crosstab(index = cat['asin'], columns="count")
quant = list(xbox_num_table.quantile([.25, .5, .75])['count'])
sample_id = cat_num_table[cat_num_table['count'] <= quant[0]].sample(n=2).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[0]) & (cat_num_table['count'] <= quant[1])].sample(n=3).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[1]) & (cat_num_table['count'] <= quant[2])].sample(n=3).index.tolist()
sample_id = sample_id + cat_num_table[cat_num_table['count'] > quant[2]].sample(n=3).index.tolist()
sample['Nintendo'] = sample_id

In [135]:
cat = relevant_data[relevant_data['category'] == 'PC']
cat_num_table = pd.crosstab(index = cat['asin'], columns="count")
quant = list(xbox_num_table.quantile([.25, .5, .75])['count'])
sample_id = cat_num_table[cat_num_table['count'] <= quant[0]].sample(n=9).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[0]) & (cat_num_table['count'] <= quant[1])].sample(n=9).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[1]) & (cat_num_table['count'] <= quant[2])].sample(n=10).index.tolist()
sample_id = sample_id + cat_num_table[cat_num_table['count'] > quant[2]].sample(n=10).index.tolist()
sample['PC'] = sample_id

In [140]:
cat = relevant_data[relevant_data['category'] == 'Sony']
cat_num_table = pd.crosstab(index = cat['asin'], columns="count")
quant = list(xbox_num_table.quantile([.5])['count'])
sample_id = cat_num_table[cat_num_table['count'] <= quant[0]].sample(n=1).index.tolist()
sample_id = sample_id + cat_num_table[cat_num_table['count'] > quant[0]].sample(n=2).index.tolist()
sample['Sony'] = sample_id

In [141]:
cat = relevant_data[relevant_data['category'] == 'PlayStation']
cat_num_table = pd.crosstab(index = cat['asin'], columns="count")
quant = list(xbox_num_table.quantile([.25, .5, .75])['count'])
sample_id = cat_num_table[cat_num_table['count'] <= quant[0]].sample(n=3).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[0]) & (cat_num_table['count'] <= quant[1])].sample(n=3).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[1]) & (cat_num_table['count'] <= quant[2])].sample(n=3).index.tolist()
sample_id = sample_id + cat_num_table[cat_num_table['count'] > quant[2]].sample(n=4).index.tolist()
sample['PlayStation'] = sample_id

In [143]:
cat = relevant_data[relevant_data['category'] == 'Wii']
cat_num_table = pd.crosstab(index = cat['asin'], columns="count")
quant = list(xbox_num_table.quantile([.25, .5, .75])['count'])
sample_id = cat_num_table[cat_num_table['count'] <= quant[0]].sample(n=1).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[0]) & (cat_num_table['count'] <= quant[1])].sample(n=2).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[1]) & (cat_num_table['count'] <= quant[2])].sample(n=2).index.tolist()
sample_id = sample_id + cat_num_table[cat_num_table['count'] > quant[2]].sample(n=2).index.tolist()
sample['Wii'] = sample_id

In [145]:
cat = relevant_data[relevant_data['category'] == 'multi_support']
cat_num_table = pd.crosstab(index = cat['asin'], columns="count")
quant = list(xbox_num_table.quantile([.33, .66])['count'])
sample_id = cat_num_table[cat_num_table['count'] <= quant[0]].sample(n=2).index.tolist()
sample_id = sample_id + cat_num_table[(cat_num_table['count'] > quant[0]) & (cat_num_table['count'] <= quant[1])].sample(n=2).index.tolist()
sample_id = sample_id + cat_num_table[cat_num_table['count'] > quant[1]].sample(n=2).index.tolist()
sample['multi_support'] = sample_id

In [146]:
print(sample)

{'xbox': ['B002TLNA7I', 'B0094MZJZ6', 'B000O5FU8I', 'B000FL8YCU', 'B001L2BF1E', 'B006Q6PYIK', 'B001FEO72M', 'B002E1SBVI', 'B00723NQO0', 'B001HWB68K'], 'More Systems': ['B0000657SO', 'B000CPKQCU', 'B0002MNXCS', 'B00000JQQJ', 'B00005NCAG', 'B00002SVFA', 'B00005ML0Z', 'B002A9JH9E', 'B0000A30RA', 'B00009WAUD', 'B00004SVV7', 'B000034DC2'], 'Nintendo': ['B0084ZY6YY', 'B000JG2DEQ', 'B002ASVYIM', 'B00JRF289K', 'B001NCJQIQ', 'B00354NBAO', 'B001QTW2HS', 'B004RLGQ56', 'B004E0M5IC', 'B004W4TG5K', 'B004EVWYYG'], 'PC': ['B000292944', 'B0000690P7', 'B002B3LHWE', 'B00004TFHM', 'B000056V4G', 'B0009YI1JO', 'B000E7XO0C', 'B004YWSKK2', 'B003EYV6ZW', 'B000124FXU', 'B004FZ97QO', '8990289432', 'B000VM0AK2', 'B006N4I3ZQ', 'B008YH3C9C', 'B00H4J647I', 'B000WNURPS', 'B001BNDPO8', 'B003NS3KBC', 'B004GKNKQQ', 'B0017WE26S', 'B008MZNRNW', 'B006VFRCOA', 'B004FYZRSM', 'B000652U0W', 'B004GWVZWU', 'B00A3DJSSE', 'B00006CRVH', 'B004JMXTNU', 'B00ELJRO1E', 'B0067STHJE', 'B00F4H2O6W', 'B00413QCS6', 'B0096PLB9O', 'B004BDBSXA'

In [150]:
sum(map(len, sample.values()))

100